In [1]:
import pandas as pd
file_path = r"C:\Users\paulb\Downloads\processed-global\global\global.tsv"

chunks= []
for chunk in pd.read_csv(file_path, sep='\t', chunksize=100000):
    chunks.append(chunk)

df = pd.concat(chunks, ignore_index=True)

In [2]:
train_path = r"C:\Users\paulb\Downloads\processed-global\global\splits\standard_tr0.9_tu0.05_te0.05_w2a93d88bac32_r2098\train.txt"
test_path = r"C:\Users\paulb\Downloads\processed-global\global\splits\standard_tr0.9_tu0.05_te0.05_w2a93d88bac32_r2098\test.txt"
val_path = r"C:\Users\paulb\Downloads\processed-global\global\splits\standard_tr0.9_tu0.05_te0.05_w2a93d88bac32_r2098\val.txt"

train_ids = pd.read_csv(train_path, header=None, sep='\t')[0]
test_ids  = pd.read_csv(test_path,  header=None, sep='\t')[0]
val_ids   = pd.read_csv(val_path,   header=None, sep='\t')[0]

train_df = df.loc[df.index.intersection(train_ids)]
test_df  = df.loc[df.index.intersection(test_ids)]
val_df   = df.loc[df.index.intersection(val_ids)]

In [3]:
def sample_per_protein(df, fraction=0.05, random_state=42):
    df['protein'] = df['pdb_fn'].str[:4]
    
    subset = df.groupby('protein', group_keys=False).apply(
        lambda x: x.sample(frac=fraction, random_state=random_state)
    )

    return subset

train_subset = sample_per_protein(train_df)
val_subset   = sample_per_protein(val_df)
test_subset  = sample_per_protein(test_df)

C:\Users\paulb\AppData\Local\Temp\ipykernel_3412\4033899297.py:4: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  subset = df.groupby('protein', group_keys=False).apply(
C:\Users\paulb\AppData\Local\Temp\ipykernel_3412\4033899297.py:4: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  subset = df.groupby('protein', group_keys=False).apply(
C:\Users\paulb\AppData\Local\Temp\ipykernel_3412\4033899297.py:4: FutureWarning: Dat

In [6]:
print(len(train_subset), len(val_subset), len(test_subset))

train_subset.head()

1249476 69415 69415


,pdb_fn,mutations,job_uuid,start_time,run_time,mutate_run_time,relax_run_time,filter_run_time,centroid_run_time,total_score,...,hs_pair,linear_chainbreak,overlap_chainbreak,pair,rg,rsigma,sheet,ss_pair,vdw,protein
25910307,1a3a_remod_p.pdb,"G58F,R107Y",in28zieAuiEa,2022-02-18 16:53:05,54,19,20,13,0,-454.290,...,4.080,0.0,0.0,-20.135,43.259,-20.208,11.670,-27.385,0.916,1a3a
23547741,1a3a_remod_p.pdb,"K6F,S136E",fU7goGx6xpDT,2022-02-12 18:43:23,51,19,18,12,0,-471.880,...,4.024,0.0,0.0,-20.134,43.213,-23.922,11.670,-32.213,1.016,1a3a
25758603,1a3a_remod_p.pdb,"K6W,L14F,T67S",VZv2MW6Q73Dx,2022-02-15 11:02:31,53,19,20,12,0,-478.609,...,5.518,0.0,0.0,-18.417,43.225,-20.492,11.670,-25.327,1.065,1a3a
23525077,1a3a_remod_p.pdb,"A19F,E39W,H111I,L118E",4pJ4nf5nhTJ7,2022-02-15 20:53:50,83,33,30,18,0,-473.181,...,2.285,0.0,0.0,-8.261,43.308,-20.259,7.894,-26.060,0.172,1a3a
25473851,1a3a_remod_p.pdb,"M1Q,T67E,G91A",Mrx8XC9VbG9p,2022-02-11 12:48:21,55,19,22,13,0,-469.566,...,4.757,0.0,0.0,-21.844,43.269,-21.863,11.670,-28.794,1.641,1a3a


In [7]:
train_subset.to_csv('train_subset.tsv', sep='\t', index=False)
val_subset.to_csv('val_subset.tsv', sep='\t', index=False)
test_subset.to_csv('test_subset.tsv', sep='\t', index=False)